# Cut My TESS Light Curve

Now that we have the TESS light curve for an entire sector, we need to cut it to individual pieces for futher analysis such as detrending them for further problems in AIJ and modelling them in EXOFAST like we have done before.

First, we need to know the reference elements ($T_0$: a reference mid-transit time, $P_{\rm orb}$: a reference orbital period) to find where the transits are in our data. A very good approximations for these reference elements based on the TESS data you are working are provided in both $dvs$ and $dvm$ files that you should have downloaded together with your TESS data. (Those pdf files in the same directory of your $FITS$ files). In those files, a mid-transit time (in TESS BJD-TDB, which is 2457000. less than the real BJD-TDB) and an orbital period are actually provided. Let's look at such a file below for our sample case HAT-P-36 b. The required reference elements are in the red box!

<br><br><br>

<center>
    <img src="hatp36_sample_dvs.png">
</center>

<br>

Now let's write these first in our code.

In [ ]:
Porb = 1.32735 # days
T0 = 1899.4769 # BTJD
# we should add 2457000. to this
T0 += 2457000.

If you can't find these or you don't have $dvs$ file for some reason (you should), you can take these values from the literature or simply from the [Exoplanet Transit Database](http://var2.astro.cz/ETD/). But please beware that these values you have taken from elsewhere might be outdated and the uncertainty on them may cause significant shifts in the predictions that you will make based on them!

Now let's open our $dvt$ file we recorded and see our light curve:

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
lcfull = pd.read_csv("HAT-P-36_TESS_DVT_lc.dat", delimiter=" ")
plt.errorbar(lcfull['time'],lcfull['flux'],lcfull['flux_err'])
plt.show()

Now we want to look at individual transits. For this we should compute a parameter called the epoch of observation as well call it in astronomy, which has a trivial formulaç It actually give you how many orbital periods that your planet covered, the integer part of which gives you the number of orbital periods and the decimal part gives you the fraction of how many periods have been taken at any given time after the reference mid-transit time ($T_0$).

$$ E = \frac{(T - T_0)}{P_{\rm orb}} $$

$T$ being the time that you would like to know the corresponding epoch. The calculation is simple in python.

In [ ]:
lcfull['epoch'] = (lcfull['time'] - T0) / Porb

In [ ]:
lcfull['epoch']

Obviously, the first transit is around 1.00, and the last transit is around 20.0 in our sample. So the integer part of the epoch tells us to which orbital cycle each point in time on our light curve corresponds to. However, we would not like to take an entire orbital cycle but rather an individual transit. The decimal part will be the key in determination of the data points we should cut. At this point, the transit duration comes into play. We will decide how many hours of the observation we should cut from the entire light curve to form an individual transit light curve based on this value too. 

We can actually calculate this value based on the information in the $dvs$ file we have taken the reference elements from. Since this relatively simple formula, which you see below, is based on transit geometry, orbital motion of the planet and relative sizes of the host star and the planet, it will be provided without further explanation for the sake of simplicity. You can find detailed information not only about this quantity but also about transit modelling in [Selcuk Yalcinkaya's seminar work](http://ozgur.astrotux.org/tezler/selcuk_yalcinkaya/seminer/selcuk_yalcinkaya_seminer_metni_final.pdf) he completed during his master's.

$$ T_{full} = \frac{P}{\pi} sin^{-1} (\frac{R_{\star}}{a} \sqrt{(1 + \frac{R_p}{R_{\star}})^2 - (\frac{a}{R_{\star}} cos~i)^2}) $$

We already know the orbital period ($P_{\rm orb}$), what we need to know to calculate this full transit duration (aka $T_{14}$ because it is the time between the first and fourth contacts of the apparent disks of the planet and the host star). Other parameters we need to know are the <b>stellar radius scaled to the semi-major axis </b> ($\frac{a}{R_{\star}}$), the radius ratio ($\frac{R_p}{R_{\star}}$), and the <b>inclination of the orbit</b> with respect to the observer's line of sight ($i$). In the $dvs$ file the first two parameters are provided (the fourth and the third row under DV Fit Results after $T_0$, $P_{\rm orb}$), this final parameter is not given but you can also calculate its value based on simple trigonometry.

$$ cos~i = b \frac{R_{\star}}{a} $$

<b>The impact parameter ($b$)</b> is also provided in the $dvs$ file. It tells you where the transit chord passes from. If it is 0, then the planet is passing through the apparent disk center of the host star, if it is 1, then it just grazes the apparent stellar disk.

Let's write these formulae in the Python language...

In [ ]:
# From dvs.pdf file
from math import asin,pi,sqrt
b = 0.02
a_Rs = 5.14
Rp_Rs = 0.1220
# Formulae to derive the full transit duration
cosi = b*(1 / a_Rs)
T_full = Porb / pi * asin((1 / a_Rs) * sqrt((1 + Rp_Rs)**2 - (a_Rs * cosi)**2))
print("Full transit duration is {:.2f} minutes".format(T_full*1440))

Obviously, 1440 appearing at the end is the number of minutes in a day because while $T_{\rm full}$ is in days, the conventional unit for the full transit duration is in minutes so that you can compare your results with that from any transiting exoplanet catalogue such as [ETD](http://var2.astro.cz/ETD/) or [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS) for your planet.

We need to convert this value to the fraction of an orbital period, so that we know how much of a cycle is covered by the transit. Then we are going to take 1.5 full transit durations before and after the transit center so that our individual light curves will cover 3 full transit transit durations, 1 of which is for the transit and 1 for each of the out-of-transit level. Considering the uncertainties on measurements, this will be a safe bet.

In [ ]:
# T14 as a fraction of an orbital period
T_full_frac = T_full / Porb
print("Full transit duration covers {:.4f} of an entire orbit".format(T_full_frac))

## Cutting The Individual Transits

Now that we are done with the calculations, we are going to cut the individual transits which will cover 3 full transit durations and save them to different files with the following code:

In [ ]:
for i,Ee in enumerate(lcfull['epoch']):
    Ee += 1.5*T_full_frac
    E = int(Ee)
    if (Ee <= (E + 3*T_full_frac)):
        fname = "tess_lc_{:d}".format(E)
        file2wr = open(fname,"a")
        str2wr = "{:.6f}\t{:.8f}\t{:.8f}\n".format(lcfull['time'][i],lcfull['flux'][i],lcfull['flux_err'][i])
        file2wr.write(str2wr)

## Visual Inspection of the Individual Transits

Now let's look at each light curve and make sure it covers the transit as we like it. Some of the transits might not be present or full due to gaps in the data. We should note and ignore these files, and then delete them to prevent future confusion.

In [ ]:
import glob
from matplotlib import pyplot as plt
fnames = glob.glob("./tess_lc_*", recursive=False)
for i,fn in enumerate(fnames):
    plt.figure(i+1)
    lc = pd.read_csv(fn, delimiter="\t",header=None)
    print("Light Curve: ", fn)
    plt.errorbar(lc[0],lc[1],lc[2])
    plt.show()
    lc.to_csv(fn, sep="\t",header=["BJD-TDB","flux","flux_err"])

You can now analyze your light curves in AIJ, detrend them there if you'd like and model them in Exofast